In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import cupy, cudf, cuml

Loading train data from .csv file

In [3]:
path = "../input/house-prices-advanced-regression-techniques/"
train_df = cudf.read_csv(path+"train.csv")

Viewing column names, non-null count and data type

In [4]:
train_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Id             1460 non-null   int64
 1   MSSubClass     1460 non-null   int64
 2   MSZoning       1460 non-null   object
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64
 5   Street         1460 non-null   object
 6   Alley          91 non-null     object
 7   LotShape       1460 non-null   object
 8   LandContour    1460 non-null   object
 9   Utilities      1460 non-null   object
 10  LotConfig      1460 non-null   object
 11  LandSlope      1460 non-null   object
 12  Neighborhood   1460 non-null   object
 13  Condition1     1460 non-null   object
 14  Condition2     1460 non-null   object
 15  BldgType       1460 non-null   object
 16  HouseStyle     1460 non-null   object
 17  OverallQual    1460 non-null   int64
 18  OverallCond    1460 non-null   

Viewing top 5 rows

In [5]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,12,2008,WD,Normal,250000


Selecting columns with float and integer data types

In [6]:
req_dtypes = ['int64', 'float64', 'int32', 'float32']
req_columns = [col for col in train_df.columns if train_df[col].dtype in req_dtypes]

In [7]:
new_train_df = train_df[req_columns]

In [8]:
new_train_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Id             1460 non-null   int64
 1   MSSubClass     1460 non-null   int64
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64
 4   OverallQual    1460 non-null   int64
 5   OverallCond    1460 non-null   int64
 6   YearBuilt      1460 non-null   int64
 7   YearRemodAdd   1460 non-null   int64
 8   MasVnrArea     1452 non-null   float64
 9   BsmtFinSF1     1460 non-null   int64
 10  BsmtFinSF2     1460 non-null   int64
 11  BsmtUnfSF      1460 non-null   int64
 12  TotalBsmtSF    1460 non-null   int64
 13  1stFlrSF       1460 non-null   int64
 14  2ndFlrSF       1460 non-null   int64
 15  LowQualFinSF   1460 non-null   int64
 16  GrLivArea      1460 non-null   int64
 17  BsmtFullBath   1460 non-null   int64
 18  BsmtHalfBath   1460 non-null   int64
 19  

Checking for null values in each column

In [9]:
new_train_df.isnull().sum()

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: uint64

Filling Null values with median of that column

In [10]:
new_train_df.fillna(new_train_df.median(), inplace=True)

In [11]:
new_train_df.isnull().sum()

Id               0
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SalePrice        0
dtype: uint64

In [12]:
X, Y = new_train_df.iloc[:, 1:-1], new_train_df.iloc[:, -1]

In [13]:
X.shape

(1460, 36)

In [14]:
Y.shape

(1460,)

Checking for moderate to high skewness

In [15]:
req_features = [col for col in X.columns if abs(X[col].skew())>0.5]

In [16]:
req_features

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'HalfBath',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

Applying log transformation to fix skewness

In [17]:
for feat in req_features:
    X[feat] = cupy.log1p(X[feat])

Standardizing data

In [18]:
std_scaler = cuml.preprocessing.StandardScaler()

X = std_scaler.fit_transform(X)

Splitting dataset

In [19]:
from cuml.preprocessing import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [20]:
from cuml.metrics.regression import r2_score, mean_absolute_error, mean_squared_error

def get_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_t = y_test.astype('float64')
    r2, mae, mse = r2_score(y_t, y_pred), mean_absolute_error(y_t, y_pred), mean_squared_error(y_t, y_pred)
    print('R2:', r2)
    print('MAE:', mae)    
    print('MSE:', mse)
    return [r2, mae, mse]

Applying svd, eig, qr, svd-qr, svd-jacobi algorithms

In [21]:
from cuml.linear_model import LinearRegression

algorithms = ["svd", "eig", 'qr', "svd-qr", "svd-jacobi"]

models = []

metrics = []

for algo in algorithms:
    model = LinearRegression(algorithm=algo)
    model.fit(X_train, y_train)
    models.append(model)
    metrics.append(get_metrics(model, X_test, y_test))


R2: 0.6706778228206118
MAE: 22925.599208738324
MSE: 1852758207.1094193
R2: 0.6706778228206123
MAE: 22925.59920873826
MSE: 1852758207.109417
R2: 0.6706778228206121
MAE: 22925.599208738306
MSE: 1852758207.1094184
R2: 0.6706778228206114
MAE: 22925.599208738335
MSE: 1852758207.1094224
R2: 0.6706778228206118
MAE: 22925.599208738324
MSE: 1852758207.1094193


In [22]:
comp_table = cudf.DataFrame(metrics, columns=['R2', 'MAE', 'MSE'], index = algorithms)

In [23]:
comp_table

,R2,MAE,MSE
svd,0.670678,22925.599209,1.852758e+09
eig,0.670678,22925.599209,1.852758e+09
qr,0.670678,22925.599209,1.852758e+09
svd-qr,0.670678,22925.599209,1.852758e+09
svd-jacobi,0.670678,22925.599209,1.852758e+09


Applying PCA to increase R2 score

In [24]:
from cuml.decomposition import PCA

pca = PCA(n_components=5)

X_pca = pca.fit_transform(X)

X_pca.shape

(1460, 5)

In [25]:
X_pca = std_scaler.fit_transform(X_pca)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.3, random_state=3)

Applying ridge regression

In [27]:
from cuml.linear_model import Ridge

algos = ["svd", "eig"]

pca_nl_models = []

pca_nl_metrics = []

for algo in algos:
    model = Ridge(alpha=1, solver=algo)
    model.fit(X_train, y_train)
    pca_nl_models.append(model)
    pca_nl_metrics.append(get_metrics(model, X_test, y_test))

R2: 0.678359658233703
MAE: 27178.126053350432
MSE: 1691102059.8261976
R2: 0.6783596582337033
MAE: 27178.126053350417
MSE: 1691102059.826197


In [28]:
ridge_stats = cudf.DataFrame(pca_nl_metrics, columns=['R2', 'MAE', 'MSE'], index=['ridge_svd', 'ridge_eig'])
ridge_stats

,R2,MAE,MSE
ridge_svd,0.67836,27178.126053,1.691102e+09
ridge_eig,0.67836,27178.126053,1.691102e+09


Applying same preprocessing steps on test data

In [29]:
t_data = cudf.read_csv(path + 'test.csv')

In [30]:
req_columns.remove('SalePrice')

In [31]:
data = t_data[req_columns].iloc[:, 1:]

In [32]:
data.isnull().sum()

MSSubClass         0
LotFrontage      227
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea        15
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       2
BsmtHalfBath       2
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       78
GarageCars         1
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: uint64

In [33]:
data.fillna(data.median(), inplace = True)

In [34]:
skew_features = [col for col in data.columns if abs(data[col].skew()) > 0.5]

In [35]:
for feature in skew_features:
    data[feature] = cupy.log1p(data[feature])

In [36]:
data = pca.fit_transform(data)
print(data.shape)

(1459, 5)


In [37]:
data = std_scaler.fit_transform(data)

In [38]:
ridge_model = pca_nl_models[0]
y_pred = ridge_model.predict(data)

In [39]:
predictions = {
    'Id': t_data['Id'].astype('int32'),
    'SalePrice': y_pred,
}
sub = cudf.DataFrame(predictions)
print(sub.info())
sub

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Id         1459 non-null   int32
 1   SalePrice  1459 non-null   float64
dtypes: float64(1), int32(1)
memory usage: 17.1 KB
None


,Id,SalePrice
0,1461,79995.281702
1,1462,221396.233954
2,1463,193686.412351
3,1464,196908.133524
4,1465,168406.778065
...,...,...
1454,2915,339525.140121
1455,2916,247091.851982
1456,2917,147152.603009
1457,2918,336895.884877


In [40]:
sub.to_csv('submission.csv', index=False)